# MIMIC-EYE 1.0 - An Eye-Tracking Data Analytics Toolkit for REFLACX and EYE GAZE

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import numpy as np
import os
import pickle as pkl
import matplotlib.pyplot as plt

In [2]:
from mimic.Patient import Patient
from mimic.XrayStudy import XrayStudy
from mimic.Xray import Xray
from MIMIC_EYE import MIMIC_EYE
from Constants import Constants as c
from data.eyegaze.BoundingBox import BoundingBox

from util.mimic_data_processing import *

In [3]:
# starting program
mimic_eye = MIMIC_EYE()

In [12]:
eye_gaze_sheet = os.path.join(c.DATASET_PATH, "spreadsheets", 'EyeGaze', "master_sheet_with_updated_stayId.csv")
eye_gaze_df = pd.read_csv( eye_gaze_sheet )

# extract all patient keys from EYE GAZE dataset
patient_keys = eye_gaze_df['patient_id'].values.tolist()[0:5]
for patient_key in patient_keys:
  
  # performs an inner join between the eye_gaze dataframe and the MIMIC core data on the patient_id key
  # this allows us to obtain all information about a patient
  # if there are multiple records of the patient (due to several visits to the hospital), we record the 
  # patient's information of his last visit to the hospital
  mimic_core = join_patients_mimic_core_data( patient_key, eye_gaze_df)

  # create a new Patient
  patient = Patient(mimic_core["patient_id"].tail(1).values[0], \
                    mimic_core["gender_x"].tail(1).values[0], \
                    mimic_core["language"].tail(1).values[0], \
                    mimic_core["marital_status"].tail(1).values[0],\
                    mimic_core["race"].tail(1).values[0], \
                    mimic_core["anchor_age_x"].tail(1).values[0], \
                    mimic_core["anchor_year"].tail(1).values[0], \
                    mimic_core["dod"].tail(1).values[0], \
                    mimic_core["anchor_year_group"].tail(1).values[0], {}, {}  )

  # get all CXR images associated to patient with patient_key
  xray_lst = []
  patient_df = eye_gaze_df[ eye_gaze_df[ "patient_id"] == patient_key ]
  for row in patient_df.itertuples():
    xray_id = row.dicom_id
    study_id = "s"+str(row.study_id)
    report = read_CXR_report( patient_key, study_id )

    # a patient can have many imagaeology studies, and each study is composed of at least one XRay image
    # one xray is given by an image file, a textual report with all the findings and diagnosis
    # and s set o
    xray_dicom_path = c.MIMIC_DICOM_PATH(c.DATASET_PATH, patient_key, study_id, xray_id)
    xray_jpg_path = c.MIMIC_JPG_PATH(c.DATASET_PATH, patient_key, study_id, xray_id)
  
    bbox_lst = []
    bboxes_df = pd.read_csv( c.EYE_GAZE_BBOX_PATH(c.DATASET_PATH, patient_key) )
    for bbox in bboxes_df.itertuples():
      bbox_lst.append(BoundingBox(bbox.x1, bbox.x2, bbox.y1, bbox.y2, bbox.bbox_name ))
    xray_lst.append( Xray(xray_id, study_id, report, xray_dicom_path, xray_jpg_path, bbox_lst))
    
  

  # find all Xrays belonging to patient with key patient_id
  
  #xrayStudy = XrayStudy(  )


  # create a new Xray
  
  


In [5]:
xray_lst

'/Volumes/SD_DISK/XAMI-MIMICv2.0/patient_15628804/CXR-DICOM/s58573295/24c7496c-d7635dfe-b8e0b87f-d818affc-78ff7cf4.dcm'

In [12]:
eye_gaze_df[ eye_gaze_df["dicom_id"] == "62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a"]

,dicom_id,path,study_id,patient_id,stay_id,gender,anchor_age,image_top_pad,image_bottom_pad,image_left_pad,image_right_pad,dx1,dx1_icd,dx2,dx2_icd,dx3,dx3_icd,dx4,dx4_icd,dx5,dx5_icd,dx6,dx6_icd,dx7,dx7_icd,dx8,dx8_icd,dx9,dx9_icd,normal_reports,Normal,CHF,pneumonia,consolidation,enlarged_cardiac_silhouette,linear__patchy_atelectasis,lobar__segmental_collapse,not_otherwise_specified_opacity___pleural__parenchymal_opacity__,pleural_effusion_or_thickening,pulmonary_edema__hazy_opacity,normal_anatomically,elevated_hemidiaphragm,hyperaeration,vascular_redistribution,atelectasis__chx,cardiomegaly__chx,consolidation__chx,edema__chx,enlarged_cardiomediastinum__chx,fracture__chx,lung_lesion__chx,lung_opacity__chx,no_finding__chx,pleural_effusion__chx,pleural_other__chx,pneumonia__chx,pneumothorax__chx,support_devices__chx,cxr_exam_indication
4,62fe5d5a-1806ee3c-f4e742fa-f2b036ea-d390057a,files/p12/p12055181/s59722264/62fe5d5a-1806ee3...,59722264,12055181,37532814.0,M,20 - 30,0,0,534,534,"Heart failure, unspecified",I50.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,NaN,1.0,NaN,0.0,NaN,NaN,NaN,1.0,NaN,1.0,NaN,-1.0,NaN,NaN,"___M with DOE, SOB in supine position // eval..."


In [5]:
patient.getID()

15628804

In [13]:
sample.columns

Index(['Unnamed: 0_x', 'patient_id', 'gender_x', 'anchor_age_x', 'anchor_year',
       'anchor_year_group', 'dod', 'Unnamed: 0_y', 'hadm_id_x', 'admittime',
       'dischtime', 'deathtime', 'admission_type', 'admission_location',
       'discharge_location', 'insurance', 'language', 'marital_status', 'race',
       'edregtime', 'edouttime', 'hospital_expire_flag', 'Unnamed: 0',
       'hadm_id_y', 'transfer_id', 'eventtype', 'careunit', 'intime',
       'outtime', 'dicom_id', 'path', 'study_id', 'stay_id', 'gender_y',
       'anchor_age_y', 'image_top_pad', 'image_bottom_pad', 'image_left_pad',
       'image_right_pad', 'dx1', 'dx1_icd', 'dx2', 'dx2_icd', 'dx3', 'dx3_icd',
       'dx4', 'dx4_icd', 'dx5', 'dx5_icd', 'dx6', 'dx6_icd', 'dx7', 'dx7_icd',
       'dx8', 'dx8_icd', 'dx9', 'dx9_icd', 'normal_reports', 'Normal', 'CHF',
       'pneumonia', 'consolidation', 'enlarged_cardiac_silhouette',
       'linear__patchy_atelectasis', 'lobar__segmental_collapse',
       'not_otherwise_speci